In [1]:
import numpy as np
from deap import base, creator, tools, algorithms
from scipy.spatial.distance import hamming

def objective_function(individual, distance_matrix, dim):
    num_vectors = distance_matrix.shape[0]
    binary_vectors = np.array(individual).reshape((dim, num_vectors)).T
    distances = np.zeros((num_vectors, num_vectors))
    
    for i in range(num_vectors):
        for j in range(num_vectors):
            if i != j:
                distances[i, j] = hamming(binary_vectors[i], binary_vectors[j])
                
    return np.sum((distances - distance_matrix)**2),

def generate_binary_vectors(dim, num_vectors, distance_matrix):
    population_size = 100
    num_generations = 100
    mutation_rate = 0.05

    # Create types
    creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
    creator.create("Individual", list, fitness=creator.FitnessMin)

    # Initialize the DEAP toolbox
    toolbox = base.Toolbox()
    toolbox.register("bit", np.random.randint, 0, 2)
    toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.bit, n=dim*num_vectors)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    
    # Register operators and evaluation function
    toolbox.register("mate", tools.cxOnePoint)
    toolbox.register("mutate", tools.mutFlipBit, indpb=mutation_rate)
    toolbox.register("select", tools.selTournament, tournsize=3)
    toolbox.register("evaluate", objective_function, distance_matrix=distance_matrix, dim=dim)

    # Generate an initial population
    population = toolbox.population(n=population_size)
    
    # Run the genetic algorithm
    result, _ = algorithms.eaSimple(population, toolbox, cxpb=1.0, mutpb=mutation_rate, ngen=num_generations, verbose=False)

    # Extract the best solution and reshape it into a matrix
    best_individual = tools.selBest(result, k=1)[0]
    best_solution = np.array(best_individual).reshape((dim, num_vectors))

    # Transpose the matrix to get the final binary vectors
    return best_solution.T


In [3]:
def similarity_error(desired_distance_matrix, calculated_distance_matrix):
    return np.sum((calculated_distance_matrix - desired_distance_matrix)**2)

def find_best_binary_vectors(dim, num_vectors, distance_matrix, num_trials):
    best_error = float("inf")
    best_vectors = None
    best_calculated_distance_matrix = None

    for trial in range(num_trials):
        binary_vectors = generate_binary_vectors(dim, num_vectors, distance_matrix)
        calculated_distance_matrix = np.zeros((num_vectors, num_vectors))

        for i in range(num_vectors):
            for j in range(num_vectors):
                if i != j:
                    calculated_distance_matrix[i, j] = hamming(binary_vectors[i], binary_vectors[j])

        error = similarity_error(distance_matrix, calculated_distance_matrix)

        if error < best_error:
            best_error = error
            best_vectors = binary_vectors
            best_calculated_distance_matrix = calculated_distance_matrix

    return best_vectors, best_calculated_distance_matrix, best_error

# Define the desired pairwise similarities between the 20 vectors
distance_matrix = np.array([
    [0, 0.2, 0.4, 0.6, 0.8],
    [0.2, 0, 0.3, 0.5, 0.7],
    [0.4, 0.3, 0, 0.2, 0.4],
    [0.6, 0.5, 0.2, 0, 0.2],
    [0.8, 0.7, 0.4, 0.2, 0]
])

# Generate binary vectors using the desired pairwise similarities
dim = 10000
num_vectors = distance_matrix.shape[0]
num_trials = 10

best_vectors, best_calculated_distance_matrix, best_error = find_best_binary_vectors(dim, num_vectors, distance_matrix, num_trials)

print("Desired pairwise similarities:")
print(distance_matrix)

print("\nBest generated binary vectors:")
print(best_vectors)

print("\nBest calculated pairwise similarities:")
print(best_calculated_distance_matrix)

print("\nBest error:")
print(best_error)


Desired pairwise similarities:
[[0.  0.2 0.4 0.6 0.8]
 [0.2 0.  0.3 0.5 0.7]
 [0.4 0.3 0.  0.2 0.4]
 [0.6 0.5 0.2 0.  0.2]
 [0.8 0.7 0.4 0.2 0. ]]

Best generated binary vectors:
[[1 1 1 ... 0 1 0]
 [0 0 0 ... 1 1 0]
 [0 0 1 ... 1 1 0]
 [0 1 1 ... 0 0 0]
 [0 1 1 ... 1 1 0]]

Best calculated pairwise similarities:
[[0.     0.4725 0.4858 0.5143 0.5191]
 [0.4725 0.     0.4875 0.5016 0.512 ]
 [0.4858 0.4875 0.     0.4695 0.4831]
 [0.5143 0.5016 0.4695 0.     0.481 ]
 [0.5191 0.512  0.4831 0.481  0.    ]]

Best error:
0.7937337199999999
